# Fake News Detector
##### O objetivo desta tarefa é praticar suas habilidades de codificação criando um sistema muito interessante: um Detector de Notícias Falsas. Aqui você tem um trecho de código que irá guiá-lo através do desenvolvimento de todo o sistema. Você só precisará alterar a parte 2. 'FakeNews' class

### Boa sorte.

##### Nome do Estudante: 
##### Estudante ID:
##### Data

# 1. Inicio
#### (você não precisa alterar essa parte)

In [1]:
# -*- coding: utf-8 -*-
#no inicio do codigo python, geralmente você adiciona todas as bibliotecas que ira usar durante o script.
# para isso basta usar "import nome_da_biblioteca"

import pandas as pd
#panda é usado para manipualação de dados.

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.metrics.pairwise import cosine_similarity
#sklearn é uma biblioteca de machine learning, e é usada para criar os modelos nesseários 
#para a classificação entre fake/true news

from nltk.corpus import stopwords
import nltk
#nltk  = natural language tool kit. é uma biblioteca para manipualao de texto.

import sys, gzip, json, datetime
import random
import numpy as np
import re
import string 
import math
from scipy.sparse import vstack
#biblioteca secundarias para manipulacao de data,valores numericos,string...

import matplotlib.pyplot as plt
# usado para plotar graficos.



In [5]:
#Nosso dataset de noticias falsas estão no arquivo mpi_snopes.csv. Então vamos usar o pandas para abrir esse arquivo 
# e usar o comando head para ver as primeiras linhas desta dataset. 

dataset= pd.read_csv("mpi_snopes.csv",encoding="utf-8")
print "Numero de linhas="+str(len(dataset))
dataset.head()

Numero de linhas=4713


,Unnamed: 0,Claim,Claim_ID,Credibility,Description,Example,Fact Check,Google Results,Last Updated,Originally Published,Origins,Referred Links,Tags,URL
0,0,"Meijer is offering $100 off ""Back to School"" c...",100-meijer-coupon,false,While each of these scams feature slight varia...,"[Collected via e-mail, August 2015] Meijer cou...","Is Meijer offering $100 off ""Back to School"" c...","[{u'results': [{u'domain': u'www.mlive.com', u...",24 August 2015,24 August 2015,"In August 2015, a survey scam promising custom...",https://www.facebook.com/meijer/photos/pb.9025...,meijer;meijer coupon;coupon;coupon scam;survey...,www.snopes.com/100-meijer-coupon/
1,1,"A photograph depicts a real (giant) pizza, par...",1000-pizza-challenge,true,"However, that linked image led only to a ""comi...","[Collected via e-mail, August 2015] I've seen ...","Does a photograph show a real (giant) pizza, p...","[{u'results': [{u'domain': u'www.snopes.com', ...",5 August 2015,5 August 2015,"In August 2015, the above-displayed image (of ...",http://www.bigmamaspizza.com/franchisee/coming...,pizza;giant pizza;pizza challenge;competitive ...,www.snopes.com/1000-pizza-challenge/
2,2,Image depicts a 102 lb. shrimp caught near Hom...,102-lb-shrimp,false,The image shows a man proudly posing with gian...,"[Collected via Facebook, November 2015]",NaN,"[{u'results': [{u'domain': u'www.snopes.com', ...",27 November 2015,27 November 2015,"On 6 November 2015, a Facebook user shared the...",http://www.livescience.com/47718-giant-alien-s...,102 lb shrimp;homosassa;fishing;,www.snopes.com/102-lb-shrimp/
3,3,A 14-year-old girl became pregnant after recei...,14-year-old-pregnant-flu-shot,false,She had all the typical symptoms of a pregnant...,"[Collected via e-mail, October 2015] I have co...",Did a 14-year-old girl become pregnant after r...,[{u'results': [{u'domain': u'worldnewsdailyrep...,5 October 2015,5 October 2015,On 23 September 2015 the web site World News D...,NaN,fake news;world news daily report;WNDR;flu sho...,www.snopes.com/14-year-old-pregnant-flu-shot/
4,4,NASA confirmed there will be 15 straight days ...,15-days-darkness-november,false,Astronomers from NASA have indicated that the ...,"[Collected via e-mail, December 2015] I can't ...",Has NASA confirmed that there will be 15 strai...,"[{u'results': [{u'domain': u'newswatch33.com',...",4 December 2015,22 July 2015,"On 14 July 2015, the web site Newswatch33 publ...",http://newswatch28.com/;https://www.nasa.gov/a...,nasa;newswatch33;huzlers;15 days of darkness;,www.snopes.com/15-days-darkness-november/


### Banco de funções
###### Aqui você vai encontrar algumas funções básicas, que serão úteis durante a criação da classe 'FakeNews', abaixo.
###### não precisa alterar nada.

In [6]:
#algumas variaveis referentes ao modelo de classificação entre fake/true news.
y = dataset.Credibility.values
X_text = dataset.Claim.values
tfidfVectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, analyzer='word',stop_words='english')
tfidfVectorizer.fit(X_text)
X = tfidfVectorizer.transform(X_text) 
model = svm.SVC(kernel='linear', C=1,probability=True).fit(X, y)


#essa função é usada para predizer se uma Afirmação(claim) é verdadeira ou falsa. Por exemplo, 'Global warming has stopped'. 
#retorno desta função é uma tupla, contendo 1) true/false; 2) probabilidade desta classe ser true/false;
#Entrada: vc pode apenas pode alterar a claim(string). 
#saida: vector of tuplas do tipo (conclusao:string, probabilidade:float)
def predictCredibility(claim,model,tfidfVectorizer):
    prediction_score=np.max(model.predict_proba(tfidfVectorizer.transform([claim])))
    prediction=model.predict(tfidfVectorizer.transform([claim]))[0]
    return [prediction,prediction_score]
#exemplo, vamos predizer se "Global warming has stopped" é verdade ou mentira.
print predictCredibility("Global warming has stopped",model,tfidfVectorizer)


#essa funcao é usada para dada uma Afirmações(claim), a funcao ira pesquisa no dataset, e retornar K entradas similares.
#Entrada: vc pode apenas pode alterar o numero K(numerico), e claim(string). 
#saida: é um vetor de tuplas. contendo 1) posição da Afirmação(claim) no nosso dataset; 2) A Claim em formato string; 3) a semelhança com a entrada
def getMostSimiarClaims(claim,tfidfVectorizer,X_text,X,quant):
    vectors_ = vstack((tfidfVectorizer.transform([claim]),X))
    #np.append(tfidfVectorizer.transform([claim]),X)
    similirity=[]
    for v in vectors_:
        #computing cosine similarity
        similirity.append(cosine_similarity(vectors_[0],v)[0][0])
    index = np.array(similirity).argsort()[-quant-1:][::-1][-quant:]
    return zip(index,X_text[index],np.array(similirity)[index])
#exemplo, obter 5 claims mais parecidas com "Global warming has stopped" do nosso dataset
print getMostSimiarClaims("Global warming has stopped",tfidfVectorizer, X_text,X,5)

[u'false', 0.76068702906962715]
[(3888, u'The U.S. government is opening gas stations to distribute free gasoline in poor neighborhoods.', 0.51882202323508442), (4099, u'Florida motorists can block the DMV from disclosing their motor vehicle and driver license records.', 0.48975879258509247), (1663, u'Urban Outfitters offered an item appearing to be a bloodstained Kent State sweatshirt for sale.', 0.21190524362869273), (3172, u'Chrysler dealerships owned by Republican donors were disproportionately targeted for closure at the behest of the Obama administration.', 0.19708275992147156), (1835, u'Someone who has taken LSD more than seven times is automatically deemed legally insane.', 0.1737216891217683)]


# 2. 'FakeNews' class
#### Essa é a parte que você deve alterar. 

In [35]:
#Usando as funções previamente definidas no acima, complete a classe a seguir. Essa classe deve ter métodos:
#1) setModel(model)
#Esse metodo ira ser reponsavel por receber nosso modelo, previamente definido. 
#2) setClaim(claim)
#Esse método simples para definir o a afirmação (claim). Você deve pegar a entrada, e difinir self.claim. Ex. self.claim=claim
#3)predictCredibilityOfClaim(claim)
#esse método deve ser reponsavel por dizer para o usuario se essa Afirmação(claim) é verdadeira ou falsa, e qual sua probabilidade.
#4)getTopKMostRelatedArticles(k)
#esse método é reponsavel por retornar para o usuario quais são as Afirmação(claim) mais parecidas.


class FakeNews:
    def __init__(self):
        self.model=None
        self.claim=None

    def setModel(self, model):
        #esse metodo já está pronta ;)
        self.model = model
        return True
    
    def setClaim(self, claim):
        #aqui completar, definindo self.claim com a entrada deste metodo.
        return True

    def predictCredibilityOfClaim(self):
        # aqui você vai usar o metodo predictCredibility. Exemplo: predictCredibility(self.claim,self.model,tfidfVectorizer)
        # Retorno deve ser uma string, exemplo: "Essa afirmação parece ser "+str(prediction[0])+ " com uma probabilidade de "+str(prediction[1])
        return True
    
    def getTopKMostRelatedArticles(self,K):
        #aqui você vai usar o metodo getMostSimiarClaims. Exemplo: getMostSimiarClaims(self.claim,tfidfVectorizer, X_text,X,5)
        #Retorno deve ser uma string, exemplo: "Existem 5 afirmação parecidas no nosso banco de dados: 1) blablabla (%semelhante), 2)... ...
        return True
            
    

# 3. Test 'FakeNews' class
### A classe que você acabou deverá ser capaz de responder corretamente retornar um valor para os seguintes métodos e entreadas. Se ela está respondendo, então está correto.
##### Aqui você não precisa alterar nada.

In [36]:
fake_news_detector = FakeNews()
fake_news_detector.setModel(model)
fake_news_detector.setClaim("Hurricane Harvey brought a shark ashore in Houston")
print fake_news_detector.predictCredibilityOfClaim()
print fake_news_detector.getTopKMostRelatedArticles(5)


True
True
